# GLACIS OpenAI Integration Demo

This notebook demonstrates how to use GLACIS to automatically attest OpenAI API calls.
All attestations are created locally in "offline" mode, using Ed25519 signatures via PyNaCl (libsodium).

## Features Demonstrated
1. Basic offline attestation with `Glacis`
2. OpenAI call interception with `attested_openai()`
3. Receipt verification
4. Crypto runtime and SQLite storage

## Setup

First, let's import the required modules and set up our signing seed.

In [ ]:
import os
import tempfile
from pathlib import Path

# Generate a random 32-byte signing seed
# In production, you'd persist this securely
SIGNING_SEED = os.urandom(32)

print(f"Signing seed (hex): {SIGNING_SEED.hex()[:32]}...")
print("Keep this seed safe - it's used to sign all attestations!")

## 1. Basic Offline Attestation

Let's start with a simple example using the `Glacis` client directly.

In [ ]:
from glacis import Glacis

# Create an offline Glacis client
glacis = Glacis(
    mode="offline",
    signing_seed=SIGNING_SEED,
)

print(f"Mode: {glacis.mode}")
print(f"Public Key: {glacis._public_key}")

In [ ]:
# Create an attestation for some data
receipt = glacis.attest(
    service_id="demo-service",
    operation_type="inference",
    input={"prompt": "What is the capital of France?"},
    output={"response": "The capital of France is Paris."},
    metadata={"model": "demo", "version": "1.0"}
)

print("=== Attestation Receipt ===")
print(f"ID: {receipt.attestation_id}")
print(f"Timestamp: {receipt.timestamp}")
print(f"Service ID: {receipt.service_id}")
print(f"Payload Hash: {receipt.payload_hash}")
print(f"Signature: {receipt.signature[:40]}...")
print(f"Public Key: {receipt.public_key}")
print(f"Witness Status: {receipt.witness_status}")

In [ ]:
# Verify the receipt
result = glacis.verify(receipt)

print("=== Verification Result ===")
print(f"Valid: {result.valid}")
print(f"Signature Valid: {result.signature_valid}")
print(f"Witness Status: {result.witness_status}")

## 2. OpenAI Integration (Non-Streaming)

Now let's use the `attested_openai()` function to automatically attest OpenAI API calls.

**Note:** You need to set your `OPENAI_API_KEY` environment variable.

In [ ]:
from glacis.integrations.openai import attested_openai, get_last_receipt

# Create an attested OpenAI client
client = attested_openai(
    openai_api_key=os.environ.get("OPENAI_API_KEY"),
    offline=True,
    signing_seed=SIGNING_SEED,
    service_id="openai-demo",
    debug=True,  # Shows attestation info
)

print("Attested OpenAI client created!")

In [ ]:
# Make a non-streaming API call
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is 2 + 2? Answer in one word."}
    ],
    max_tokens=10,
)

print("=== OpenAI Response ===")
print(f"Model: {response.model}")
print(f"Content: {response.choices[0].message.content}")

In [ ]:
# Get the attestation receipt
receipt = get_last_receipt()

print("=== Attestation Receipt ===")
print(f"ID: {receipt.attestation_id}")
print(f"Service ID: {receipt.service_id}")
print(f"Operation: {receipt.operation_type}")
print(f"Payload Hash: {receipt.payload_hash}")
print(f"Witness Status: {receipt.witness_status}")

## 3. Crypto Runtime Demo

Under the hood, GLACIS uses PyNaCl (libsodium bindings) for cryptographic operations.
Let's explore the crypto runtime directly.

In [ ]:
from glacis.wasm_runtime import WasmRuntime

# Get the singleton runtime instance (uses PyNaCl by default)
runtime = WasmRuntime.get_instance()
print(f"Runtime type: {type(runtime).__name__}")

# SHA-256 hashing
message = b"Hello, GLACIS!"
hash_result = runtime.sha256(message)
print(f"\nSHA-256 of '{message.decode()}':")
print(f"  {hash_result.hex()}")

# Derive public key from seed
pubkey = runtime.derive_public_key(SIGNING_SEED)
print(f"\nPublic Key (from seed):")
print(f"  {pubkey.hex()}")

# Sign a message
signature = runtime.ed25519_sign(SIGNING_SEED, message)
print(f"\nEd25519 Signature:")
print(f"  {signature.hex()[:64]}...")

# Verify the signature
is_valid = runtime.ed25519_verify(pubkey, message, signature)
print(f"\nSignature Valid: {is_valid}")

## 4. SQLite Storage

Offline receipts are persisted to SQLite for durability.

In [ ]:
# Query receipts from storage
all_receipts = glacis._storage.query_receipts(limit=10)

print(f"=== Stored Receipts ({len(all_receipts)} found) ===")
for r in all_receipts:
    print(f"  {r.attestation_id} | {r.service_id} | {r.timestamp}")

In [ ]:
# Get the last receipt from storage
last_receipt = glacis.get_last_receipt()

if last_receipt:
    print("=== Last Receipt ===")
    print(f"ID: {last_receipt.attestation_id}")
    print(f"Timestamp: {last_receipt.timestamp}")
else:
    print("No receipts found in storage.")

## Summary

GLACIS provides transparent attestation for AI operations:

- **Offline Mode**: All cryptography happens locally via PyNaCl (libsodium)
- **Ed25519 Signatures**: Tamper-evident receipts
- **SQLite Persistence**: Durable local storage
- **OpenAI Integration**: Automatic attestation for completions
- **Witness Status**: `UNVERIFIED` for offline, can be witnessed later

Each receipt contains:
- `attestation_id`: Unique identifier (oatt_xxx for offline)
- `payload_hash`: SHA-256 of canonical JSON (input + output)
- `signature`: Ed25519 signature over the payload
- `public_key`: Derived from signing seed
- `witness_status`: `UNVERIFIED` until submitted to transparency log

In [ ]:
# Cleanup
glacis.close()
print("Demo complete!")